In [2]:
import rateslib as rl

In [3]:
eureur = rl.Curve(
    nodes={
        rl.dt(2022, 1, 1): 1.0,
        rl.dt(2022, 5, 1): 1.0,
        rl.dt(2023, 1, 1): 1.0,
    },
    convention="act360",
    calendar="tgt",
    interpolation="log_linear",
    id="eureur",
)


eur_kws = dict(
    effective=rl.dt(2022, 1, 3),
    spec="eur_irs",
    curves="eureur",
)


eur_solver = rl.Solver(
    curves=[eureur],
    instruments=[
        rl.IRS(**eur_kws, termination="4M"),
        rl.IRS(**eur_kws, termination="1Y"),
    ],
    s=[2.0, 2.5],
    id="eur",
)

SUCCESS: `func_tol` reached after 3 iterations (levenberg_marquardt) , `f_val`: 1.1522476203957725e-18, `time`: 0.0070s


In [4]:
usdusd = rl.Curve(
    nodes={
        rl.dt(2022, 1, 1): 1.0,
        rl.dt(2022, 8, 1): 1.0,
        rl.dt(2023, 1, 1): 1.0,
    },
    convention="act360",
    calendar="nyc",
    interpolation="log_linear",
    id="usdusd",
)


usd_kws = dict(
    effective=rl.dt(2022, 1, 3),
    spec="usd_irs",
    curves="usdusd",
)


usd_solver = rl.Solver(
    curves=[usdusd],
    instruments=[
        rl.IRS(**usd_kws, termination="7M"),
        rl.IRS(**usd_kws, termination="1Y"),
    ],
    s=[4.0, 4.8],
    id="usd",
)

SUCCESS: `func_tol` reached after 3 iterations (levenberg_marquardt) , `f_val`: 8.95311645138308e-13, `time`: 0.0040s


In [5]:
fxr = rl.FXRates({"eurusd": 1.10}, settlement=rl.dt(2022, 1, 3))

eurusd = rl.Curve(
    nodes={
        rl.dt(2022, 1, 1): 1.0,
        rl.dt(2022, 5, 1): 1.0,
        rl.dt(2022, 9, 1): 1.0,
        rl.dt(2023, 1, 1): 1.0,
    },
    convention="act360",
    calendar=None,
    interpolation="log_linear",
    id="eurusd",
)


fxf = rl.FXForwards(
    fx_rates=fxr,
    fx_curves={
        "usdusd": usdusd,
        "eureur": eureur,
        "eurusd": eurusd,
    },
)


xcs_kws = dict(
    effective=rl.dt(2022, 1, 3),
    spec="eurusd_xcs",
    curves=["eureur", "eurusd", "usdusd", "usdusd"],
)


xcs_solver = rl.Solver(
    pre_solvers=[eur_solver, usd_solver],
    fx=fxf,
    curves=[eurusd],
    instruments=[
        rl.XCS(**xcs_kws, termination="4m"),
        rl.XCS(**xcs_kws, termination="8m"),
        rl.XCS(**xcs_kws, termination="1y"),
    ],
    s=[-5.0, -6.5, -11.0],
    id="eur/usd",
)

SUCCESS: `func_tol` reached after 3 iterations (levenberg_marquardt) , `f_val`: 1.0572444443750555e-20, `time`: 0.0530s
